In [3]:
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os, sys, shutil, importlib, glob
from tqdm import tqdm_notebook as tqdm
from tqdm.notebook import tqdm
from celloracle import motif_analysis as ma
from celloracle.utility import save_as_pickled_object
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
plt.rcParams['figure.figsize'] = [6, 4.5]
plt.rcParams["savefig.dpi"] = 300

/home/ekernf01/.local/share/r-miniconda/envs/cell_type_grn_transfer/lib/python3.9/site-packages/celloracle/trajectory/markov_simulation.py:17: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "_walk" failed type inference due to: Undecided type $22load_method.7 := <undecided>
During: resolving caller type: $22load_method.7
During: typing of call at /home/ekernf01/.local/share/r-miniconda/envs/cell_type_grn_transfer/lib/python3.9/site-packages/celloracle/trajectory/markov_simulation.py (40)


File "../../../../../../../../.local/share/r-miniconda/envs/cell_type_grn_transfer/lib/python3.9/site-packages/celloracle/trajectory/markov_simulation.py", line 40:
def _walk(start_cell_id_array, transition_prob, n_steps):
    <source elided>

    li.append(list(ids_now))
    ^

  @jit(i8[:,:](i8[:], f8[:,:], i8))
/home/ekernf01/.local/share/r-miniconda/envs/cell_type_grn_transfer/lib/python3.9/site-packages/celloracle/trajectory/markov_simulation.

In [ ]:
# Load bed_file
file_path_of_bed_file = "bulk_ATAC_seq_peak_data.bed" 
bed = ma.read_bed(file_path_of_bed_file)
print(bed.shape)
bed.head()

# Convert bed file into peak name list
peaks = ma.process_bed_file.df_to_list_peakstr(bed)
peaks

# Make TSS annotation
tss_annotated = ma.get_tss_info(peak_str_list=peaks, ref_genome=???)

# Change format
peak_id_tss = ma.process_bed_file.df_to_list_peakstr(tss_annotated)
tss_annotated = pd.DataFrame({"peak_id": peak_id_tss,
                              "gene_short_name": tss_annotated.gene_short_name.values})
tss_annotated = tss_annotated.reset_index(drop=True)
tss_annotated.to_csv("processed_peak_file.csv")

In [ ]:


plt.rcParams['figure.figsize'] = (15,7)
plt.rcParams["savefig.dpi"] = 600
1. Rerefence genome data preparation
Before starting the analysis, we need to make sure the reference genome data is installed. If not, please install the correct reference genome using the instructions below.

# PLEASE make sure reference genome is correct.
ref_genome = "mm10"

genome_installation = ma.is_genome_installed(ref_genome=ref_genome)
print(ref_genome, "installation: ", genome_installation)
mm10 installation:  True
if not genome_installation:
    import genomepy
    genomepy.install_genome(ref_genome, "UCSC")
else:
    print(ref_genome, "is installed.")
    
    
    


In [ ]:
# Instantiate TFinfo object
tfi = ma.TFinfo(peak_data_frame=peaks,
                ref_genome=ref_genome)


%%time
# Scan motifs. !!CAUTION!! This step may take several hours if you have many peaks!
tfi.scan(fpr=0.02,
         motifs=None,  # If you enter None, default motifs will be loaded.
         verbose=True)

# Save tfinfo object
tfi.to_hdf5(file_path="test1.celloracle.tfinfo")

# Reset filtering
tfi.reset_filtering()

# Do filtering
tfi.filter_motifs_by_score(threshold=10)

# Format post-filtering results.
tfi.make_TFinfo_dataframe_and_dictionary(verbose=True)

# Save result as a dataframe
df = tfi.to_dataframe()
df.to_parquet(os.path.join(folder, "base_GRN_dataframe.parquet"))
